In [1]:
import urllib
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
headers = {'User-agent': 'Mozilla/5.0'}

In [3]:
formula1_home = 'https://resultados.as.com/resultados/motor/formula_1/{}/clasificacion'

In [4]:
def count_columns(table_datos):
    rv = 0
    for tr in table_datos.find("tr"):
        try:
            is_column = tr['scope']
            if is_column == 'col':
                rv += 1
        except:
            pass
    return rv

In [5]:
def lista_into_chunks(lista, chunk_size):
    chunk_size = max(1, chunk_size)
    return [lista[i: i + chunk_size] for i in range(0, len(lista), chunk_size)]

In [6]:
pilotos_puntos = {}
for i in range(2008, 2020):
    pilotos_puntos[i] = {}
    url = formula1_home.format(i)    
    html = requests.get(url, headers=headers)
    soup = BeautifulSoup(html.content, "lxml")
    table_datos = soup.find("table", { "class" : "tabla-datos" })
    num_cols = count_columns(table_datos)
    tds = table_datos.findAll('td')
    
    # extrae pilotos
    contador_piloto = 1
    for row in table_datos.findAll('tr'):
        row_span = row.findAll('span')
        if len(row_span) > 0:
            result = []
            for x in row_span:
                result.append(x.get_text())
            pilotos_puntos[i][contador_piloto] = [result[1]]
            contador_piloto += 1
    
    # extrae puntos
    tds_chunks = lista_into_chunks(tds, num_cols - 1)
    contador_chunk = 1
    for chunk in tds_chunks:
        for x in chunk: 
            try:
                x_value = int(x.get_text())
            except:
                x_value = '-'
            pilotos_puntos[i][contador_chunk].append(x_value)
        contador_chunk += 1

In [11]:
tabla = pd.DataFrame(pilotos_puntos)

In [12]:
print(tabla)

                                                 2008  \
1   [L. Hamilton, 98, 10, 4, 0, 6, 8, 10, 0, 0, 10...   
2   [F. Massa, 97, 0, 0, 10, 8, 10, 6, 4, 10, 0, 6...   
3   [K. Raikkonen, 75, 1, 10, 8, 10, 6, 0, 0, 8, 5...   
4   [Robert Kubica, 75, 0, 8, 6, 5, 5, 8, 10, 4, 0...   
5   [F. Alonso, 61, 5, 1, 0, 0, 3, 0, 0, 1, 3, 0, ...   
6   [\nNick Heidfeld\n, 60, 8, 3, 5, 0, 4, 0, 8, 0...   
7   [\nHeikki Kovalainen\n, 53, 4, 6, 4, 0, 0, 1, ...   
8   [S. Vettel, 35, 0, 0, 0, 0, 0, 4, 1, 0, 0, 1, ...   
9   [\nJarno Trulli\n, 31, 0, 5, 3, 1, 0, 0, 3, 6,...   
10  [\nTimo Glock\n, 25, 0, 0, 0, 0, 0, 0, 5, 0, 0...   
11  [\nM. Webber\n, 21, 0, 2, 2, 4, 2, 5, 0, 3, 0,...   
12  [\nNelsinho Piquet\n, 19, 0, 0, 0, 0, 0, 0, 0,...   
13  [N. Rosberg, 17, 6, 0, 1, 0, 1, 0, 0, 0, 0, 0,...   
14  [\nRubens Barrichello\n, 11, 0, 0, 0, 0, 0, 3,...   
15  [\nKazuki Nakajima\n, 9, 3, 0, 0, 2, 0, 2, 0, ...   
16  [\nDavid Coulthard\n, 8, 0, 0, 0, 0, 0, 0, 6, ...   
17  [\nSebastien Bourdais\n, 4,

In [14]:
tabla.to_excel(r'C:\Users\joaco\Econometrics\Logit F1\Resultados.xlsx', index = False)